1. Install Required Libraries

In [1]:
pip install transformers torch pandas scikit-learn

In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [19]:
pip install openpyxl


In [22]:
import pandas as pd

# Load the Excel dataset
df = pd.read_excel('/content/hi.xlsx')


In [23]:
print(df.head())



                                            sentence sentiment
0  I love the decks in this game, there’s a great...  positive
1        I know these games aren’t meant to be easy.  negative
2  more decks and couldn’t get it to load despite...  negative
3                Then I try to get my old save back.  positive
4  Instead, the game is a candy crush style game ...  positive


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode labels if they are categorical (like 'positive', 'negative')
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['sentence'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42
)


In [25]:
from transformers import BertTokenizer

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [27]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Initialize BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['sentiment'].unique()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,No log,0.529751
2,No log,0.724488
3,0.364000,0.877893


TrainOutput(global_step=600, training_loss=0.3338241736094157, metrics={'train_runtime': 3013.2051, 'train_samples_per_second': 1.592, 'train_steps_per_second': 0.199, 'total_flos': 98604979106400.0, 'train_loss': 0.3338241736094157, 'epoch': 3.0})

In [29]:
# Evaluate on test set
results = trainer.evaluate()
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 0.8778933882713318, 'eval_runtime': 66.1942, 'eval_samples_per_second': 6.043, 'eval_steps_per_second': 0.755, 'epoch': 3.0}


In [31]:
def predict(sentences):
    # Tokenize input sentences
    encodings = tokenizer(sentences, truncation=True, padding=True, max_length=128, return_tensors="pt")
    # Forward pass to get predictions
    outputs = model(**encodings)
    # Get predicted class (highest logit)
    predictions = torch.argmax(outputs.logits, dim=-1)
    # Convert predicted class index back to original sentiment labels
    return label_encoder.inverse_transform(predictions.numpy())

# Example usage
sentences = ["This game is fantastic!"]
print("Predictions:", predict(sentences))


Predictions: ['positive']
